<a href="https://colab.research.google.com/github/Garima-is-awesome/Automatic_Speech_Recognition/blob/main/Sequence_Classification_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pickle
import random

n_train = 10000
n_test = 100

train_samples = []
train_labels = []
test_samples = []
test_labels = []

for i in range(n_train + n_test):
  if i < n_train:
    samples = train_samples
    labels = train_labels
  else:
    samples = test_samples
    labels = test_labels

  label = random.choice([0, 1])
  start = random.uniform(-40.0, 40.0)
  sample = [start]
  next = start + random.uniform(-1, 5.0) * (-label * 2 + 1)
  sample.append(next)
  next = next + random.uniform(-2, 10.0) * (-label * 2 + 1)
  sample.append(next)
  next = next + random.uniform(-2, 10.0) * (-label * 2 + 1)
  sample.append(next)


  print("label:", label)
  print("sample:", sample)
  labels.append(label)
  samples.append(sample)

  data = {
    "train_samples": train_samples,
    "train_labels": train_labels,
    "test_samples": test_samples,
    "test_labels": test_labels,
  }

with open('data.pickle', 'wb') as f:
  pickle.dump(data, f)


Streaming output truncated to the last 5000 lines.
label: 0
sample: [36.406737404555756, 41.27650243853715, 47.74762966648255, 57.255545651768024]
label: 1
sample: [-24.665387823482547, -25.691052967561834, -31.17333511352598, -34.03107209501542]
label: 1
sample: [3.1076579265902, 0.39645322593128984, 1.406537499368417, 3.39863068583484]
label: 0
sample: [39.478127010045355, 44.41569728749633, 44.25605618015628, 48.122103148182404]
label: 0
sample: [-9.394601496829864, -4.629276002215909, 0.4803183138148528, -0.8020997391182414]
label: 0
sample: [-32.91578854320352, -32.46361955623383, -26.32087146573036, -18.98133714127354]
label: 0
sample: [25.20952193428164, 29.836067606147626, 36.58183889027083, 46.02946570533385]
label: 0
sample: [39.42723454938435, 42.73542233885536, 47.92746731008038, 50.427019199804846]
label: 0
sample: [26.606724879679916, 27.60414731231384, 33.152553283176736, 31.35084968408921]
label: 1
sample: [-28.983841358913878, -29.04804551859403, -34.82651642446717, -3

In [2]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim


def GetData():
  with open('data.pickle', mode='rb') as f:
    data = pickle.load(f)

  train_samples = np.array(data['train_samples'])
  train_labels = np.array(data['train_labels'])
  test_samples = np.array(data['test_samples'])
  test_labels = np.array(data['test_labels'])
  return train_samples, train_labels, test_samples, test_labels

class MyNet(nn.Module):
  def __init__(self):
    super(MyNet, self).__init__()
    # Define the RNN layer and a final linear layer
    self.rnn = nn.RNN(
      input_size=1,
      hidden_size=8,
      num_layers=1)
    self.feedforward_output = nn.Linear(
      in_features=8,
      out_features=1)

  def forward(self, x):
    h0 = torch.zeros(1, x.shape[1], 8)
    y, hn = self.rnn(x, h0)
    y = torch.sigmoid(self.feedforward_output(y))
    return y


def main():
  train_samples, train_labels, test_samples, test_labels = GetData()

  mynet = MyNet()

  # Train
  criterion = nn.BCELoss()
  optimizer = optim.Adam(mynet.parameters(), lr=0.1)
  num_steps = 100
  print('start training')
  for epoch in range(num_steps):
      optimizer.zero_grad()
      inputs = np.expand_dims(train_samples.transpose(), -1)
      outputs = torch.squeeze(
          mynet(torch.from_numpy(inputs).float()))
      last_outputs = outputs[-1, :]
      loss = criterion(
          last_outputs, torch.from_numpy(train_labels).float())
      loss.backward()
      optimizer.step()
      print('epoch:', epoch, 'loss:', loss.item())
  print('finished training')

  # Evaluate
  inputs = np.expand_dims(test_samples.transpose(), -1)
  test_predict = np.squeeze(mynet(torch.from_numpy(inputs).float()).data.numpy())
  last_predict = (test_predict[-1, :] > 0.5)
  accuracy = sum(last_predict == test_labels) / len(test_labels)
  print('accuracy on test data:', accuracy)

if __name__ == '__main__':
  main()

start training
epoch: 0 loss: 0.7392510175704956
epoch: 1 loss: 0.6407977342605591
epoch: 2 loss: 0.6121233105659485
epoch: 3 loss: 0.5590617656707764
epoch: 4 loss: 0.568320095539093
epoch: 5 loss: 0.5547086000442505
epoch: 6 loss: 0.4839553236961365
epoch: 7 loss: 0.4846881926059723
epoch: 8 loss: 0.46270009875297546
epoch: 9 loss: 0.4356590807437897
epoch: 10 loss: 0.38720086216926575
epoch: 11 loss: 0.36089888215065
epoch: 12 loss: 0.36179232597351074
epoch: 13 loss: 0.33851927518844604
epoch: 14 loss: 0.30238205194473267
epoch: 15 loss: 0.3572295308113098
epoch: 16 loss: 0.34886863827705383
epoch: 17 loss: 0.37051185965538025
epoch: 18 loss: 0.3154074251651764
epoch: 19 loss: 0.3702147603034973
epoch: 20 loss: 0.3008003532886505
epoch: 21 loss: 0.27412110567092896
epoch: 22 loss: 0.256210595369339
epoch: 23 loss: 0.2808813750743866
epoch: 24 loss: 0.2509704828262329
epoch: 25 loss: 0.24345813691616058
epoch: 26 loss: 0.26172390580177307
epoch: 27 loss: 0.24626514315605164
epoch: 2

In [5]:

print(data)

{'train_samples': [[12.546767092374424, 16.093371509227104, 23.442768196601666, 23.25839692021666], [-25.919228127703285, -29.579695317143386, -27.94719529268648, -37.030921305537575], [2.2724398082576798, 7.041814454152216, 13.56648610312945, 19.72531132575863], [3.3430768897454826, 6.931134802337327, 8.713392473620669, 15.226300199222724], [14.597113768704503, 18.557207852248972, 27.804298805734447, 31.040187120282162], [0.9687257640852209, 0.29924271692043924, -3.6313551404652475, -11.712951729100578], [18.592336703968726, 17.83341745186816, 13.6613209925235, 6.022496281732794], [13.959934102661023, 13.76801457383134, 14.784363294345436, 16.965572796353744], [-13.424087174485972, -8.989879944427294, -9.26425083731688, -7.188107130703504], [-30.014093930013168, -34.76012552298631, -41.718899124237666, -42.4305892909038], [-32.4484647296688, -30.75600296013974, -25.531715796971085, -16.42259655197634], [19.319631246070514, 15.540302458904623, 14.553996254712793, 15.682497467289561], [